# 데이터 전처리 EX-3
## Diamonds Dataset을 이용한 가격 예측 모델 구현 (회귀 모델)
### 문제 3: 이상치 제거 (IQR 이용)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# 1. 데이터 불러오기
df = pd.read_csv('diamonds.csv')

# 2. carat열의 IQR 계산
Q1 = df['carat'].quantile(0.25)
Q3 = df['carat'].quantile(0.75)
IQR = Q3 - Q1

# 3. 이상치 경계 계산 후 하한과 상한 값 출력
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

print(f"IQR: {IQR}")
print(f"하한 (lower bound): {lower_bound}")
print(f"상한 (upper bound): {upper_bound}")

# 4. carat 열에서 IQR 방식을 사용하여 이상치 제거
df_no_outliers = df[(df['carat'] >= lower_bound) & (df['carat'] <= upper_bound)]

# 5. 이상치 제거 후 데이터의 개수를 출력
print(f"원래 데이터의 개수: {len(df)}")
print(f"이상치 제거 후 데이터의 개수: {len(df_no_outliers)}")

# 6. 특성과 라벨 분리
X = df.drop('price', axis=1)
y = df['price']

# 7. 수치형 / 범주형 특징을 구분
num_data = ['carat', 'depth', 'table', 'x', 'y', 'z']
ctgy_data = ['cut', 'color', 'clarity']

# 8. 전처리 구성
num_trans = StandardScaler()
ctgy_trans = OneHotEncoder(drop='first', handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_trans, num_data),
        ('ctgy', ctgy_trans, ctgy_data)
    ]
)

# 9. 파이프라인 구성
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

# 10. 학습/테스트 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 11. 모델 학습
model.fit(X_train, y_train)

# 12. 예측
y_pred = model.predict(X_test)

# 13. 평가
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("\n모델 성능")
print(f"mse: {mse:.2f}")
print(f"rmse: {rmse:.2f}")
print(f"mae: {mae:.2f}")
print(f"r2 Score: {r2:.4f}")

IQR: 0.64
하한 (lower bound): -0.5599999999999999
상한 (upper bound): 2.0
원래 데이터의 개수: 53940
이상치 제거 후 데이터의 개수: 52051

모델 성능
mse: 1288705.48
rmse: 1135.21
mae: 737.15
r2 Score: 0.9189


##### MSE 값이 작을수록 모델이 더 정확하다. 
##### RMSE는 평균적으로 실제 값과 1135 오차가 난다고 본다.
##### MAE는 평균적으로 실제 값과 737정도 차이난다고 본다.
##### R2는 모델은 데이터를 약 91.89% 설명한다고 본다.
##### R2가 약 92%인 것으로 보아 모델은 잘 작동하지만
##### 1000(price)달러 이상의 가격 예측 오차는 다이아몬드를 구매하는데 큰 문제라고 볼 수 있다. 